# 1. Importing libraries

In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training.")

Using cpu for training.


# 2. Loading data & Preprocessing
- 데이터 로드
  - 종별로 1,000장씩 총 7종 - 사진 7,000장
  - train_test_split 이용하여 8:2로 분할 
- 데이터 전처리

In [2]:
# 데이터 로드 및 분할
data = pd.read_csv('datasetForAWS.csv')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

for label in data['라벨'].unique():
    label_data = data[data['라벨'] == label]
    train_label_data, test_label_data = train_test_split(label_data, test_size=200, random_state=42)  # Splitting
    train_data = train_data.append(train_label_data)
    test_data = test_data.append(test_label_data)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

print(train_data[:10])

                           이미지_경로        라벨
0   img/elephant/elephant.213.jpg  elephant
1           img/lamb/lamb.327.jpg      lamb
2  img/elephant/elephant.453.jpeg  elephant
3  img/squirrel/squirrel.845.jpeg  squirrel
4  img/squirrel/squirrel.580.jpeg  squirrel
5          img/lamb/lamb.870.jpeg      lamb
6            img/cat/cat.4520.jpg       cat
7            img/dog/dog.4705.jpg       dog
8  img/squirrel/squirrel.259.jpeg  squirrel
9  img/elephant/elephant.825.jpeg  elephant


/tmp/ipykernel_17097/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
/tmp/ipykernel_17097/994303206.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
/tmp/ipykernel_17097/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
/tmp/ipykernel_17097/994303206.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
/tmp/ipykernel_17097/994303206.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [3]:
# 사용자 정의 데이터셋 클래스
class AnimalDataset(Dataset):
    def __init__(self, dataframe, label_map, transform=None):
        self.dataframe = dataframe
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        image = Image.open(image_path).convert("RGB")  # tensor 단위 오류 해결 위해 명시
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_map[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# 라벨을 정수로 매핑 - 추후 label을 숫자가 아닌 글자로 반환하도록 해주는 딕셔너리
label_map = {'cat': 0, 'cow': 1, 'dog': 2, 'elephant': 3, 'horse': 4, 'lamb': 5, 'squirrel': 6}

# 이미지 전처리를 위한 transform 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 및 데이터 로더 생성
train_dataset = AnimalDataset(train_data, label_map, transform=transform)  # 인스턴스는 init 파라미터 따름
test_dataset = AnimalDataset(test_data, label_map, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=48, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=48, shuffle=False, num_workers=2)

# 3. Training

In [4]:
# 모델 정의
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # imageNet 데이터로 훈련된 모델 불러옴
"""
Q. 왜 내 모델에 사용할 이미지도 아닌 imageNet 이미지로 학습한 모델을 불러오는가?
A. 이미 한 번 이미지 분류를 학습한 모델을 불러오면 복잡한 학습 과정에 들어가는 시간과 비용 절감 가능
   (가중치를 조금만 수정하면 되니까) 
"""
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7종의 동물 클래스
model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
        
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
    
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    return model

In [5]:
# 모델 학습
trained_model = train_model(model, criterion, optimizer, num_epochs=5)

Epoch 1/5:   4%|▍         | 5/117 [00:53<19:57, 10.69s/it]


KeyboardInterrupt: 

# 4. Testing

In [27]:
# 테스트 함수
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'테스트 데이터에 대한 모델 정확도: {accuracy:.2f}%')

# 모델 테스트
test_model(trained_model, test_loader)

테스트 데이터에 대한 모델 정확도: 72.14%


# 4. Saving Model

In [28]:
# 현재 디렉토리에 모델 저장
torch.save(model.state_dict(), 'modelv1_acc72.pth')

# 5. Recall

In [29]:
# 모델 정의 (동일한 모델 구조 필요)
model = models.resnet18(weights=None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 클래스 수에 맞게 조정

# 상태 사전 로드
model.load_state_dict(torch.load('modelv1_acc72.pth'))

# 모델을 평가 모드로 설정
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# TODO 1 : 학습 속도 절감
# TODO 2 : 모델 정확도 개선 - 적절한 성능 지표, 하이퍼파라미터 등
# TODO 3 : 주로 영향을 미치는 하이퍼파라미터 정리